In [195]:
from scipy.optimize import linprog
from math import sqrt
import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt 
import timeit
import pandas as pd
from scipy import stats
import time

In [196]:
random.seed(42)

In [285]:
# hardcoded constants

# Scaling factor for alpha, beta to set new prices
alpha_scaling_factor = 0.5
beta_scaling_factor = budget/50

num_subjects = 861
num_treatments = 2
capacity_matrix = [400, 461]
budget = 10
budget_matrix = [budget] * num_subjects

In [286]:
pte_df = pd.read_csv("PTE.csv")
# pte_df.head()

wtp_df = pd.read_csv("WTP.csv")
# wtp_df.head()

pte_matrix = #[[0, i] for i in pte_df['PTE'].values.tolist()]
wtp_matrix = #[[0, i] for i in wtp_df['WTP'].values.tolist()]

In [300]:
# Willingness to pay matrix w(i,t). Dimensions num_subjects * num_treatments
wtp_matrix = [[random.randint(1, 100) for i in range(num_treatments)] for x in range(num_subjects)]
print "Willingness to pay matrix:", wtp_matrix

Willingness to pay matrix: [[77, 68], [49, 58], [27, 42], [46, 64], [89, 10], [52, 28], [94, 37], [96, 33], [1, 78], [74, 74], [46, 67], [36, 7], [54, 22], [43, 22], [27, 83], [34, 58], [57, 49], [35, 69], [5, 10], [79, 46], [13, 86], [45, 1], [96, 21], [69, 14], [65, 16], [94, 28], [66, 26], [38, 91], [17, 40], [31, 70], [24, 66], [71, 1], [48, 14], [23, 68], [1, 70], [82, 99], [43, 14], [8, 39], [74, 11], [32, 89], [14, 78], [76, 14], [100, 15], [54, 1], [66, 45], [73, 63], [16, 42], [69, 86], [9, 11], [76, 59], [39, 97], [32, 14], [28, 9], [56, 61], [61, 78], [70, 85], [66, 31], [52, 51], [75, 30], [6, 90], [96, 50], [12, 50], [60, 53], [98, 99], [94, 14], [87, 57], [37, 69], [77, 96], [78, 2], [7, 27], [4, 7], [79, 51], [63, 51], [42, 71], [9, 54], [62, 28], [31, 52], [21, 81], [54, 40], [64, 84], [69, 7], [70, 73], [85, 6], [9, 44], [46, 61], [31, 75], [75, 12], [71, 71], [17, 96], [53, 79], [73, 17], [13, 79], [27, 89], [78, 3], [81, 28], [7, 72], [58, 8], [46, 37], [50, 57], [37

In [301]:
# Predicted treatment effect e(i,t). Dimensions num_subjects * num_treatments
pte_matrix = [[random.randint(1, 100) for i in range(num_treatments)] for x in range(num_subjects)]
print "Predicted treatment effect matrix:", pte_matrix

Predicted treatment effect matrix: [[38, 86], [28, 18], [14, 58], [23, 10], [27, 24], [44, 39], [15, 96], [15, 81], [18, 50], [100, 85], [52, 73], [79, 31], [57, 57], [40, 70], [7, 82], [48, 63], [45, 34], [37, 57], [94, 26], [2, 13], [87, 97], [20, 58], [65, 18], [79, 36], [68, 49], [74, 89], [39, 29], [64, 15], [17, 81], [34, 64], [58, 85], [8, 17], [23, 32], [30, 27], [65, 28], [45, 87], [37, 59], [97, 42], [19, 3], [73, 67], [95, 71], [9, 17], [11, 7], [88, 55], [3, 40], [77, 9], [27, 16], [66, 89], [31, 25], [29, 63], [14, 84], [3, 67], [87, 33], [48, 98], [55, 28], [45, 97], [70, 18], [60, 64], [64, 57], [53, 64], [31, 35], [54, 81], [45, 37], [26, 31], [1, 82], [85, 35], [47, 1], [93, 95], [48, 1], [44, 30], [24, 1], [38, 42], [57, 40], [17, 74], [39, 38], [27, 43], [24, 77], [91, 81], [69, 29], [75, 81], [42, 86], [19, 29], [64, 62], [28, 63], [19, 2], [5, 54], [19, 11], [27, 72], [73, 24], [16, 50], [35, 32], [80, 100], [47, 80], [34, 85], [96, 6], [78, 8], [48, 20], [85, 82],

In [302]:
# Convert lists to np.array type
wtp_matrix = np.array(wtp_matrix)
pte_matrix = np.array(pte_matrix)
budget_matrix = np.array(budget_matrix)
capacity_matrix = np.array(capacity_matrix)

In [303]:
# Init alpha, beta assumed to be positive
def init_alpha():
    alpha = [random.randint(-budget, 0) for i in range(num_treatments)]
    return alpha
def init_beta():
    beta = [random.randint(-budget, budget) for i in range(num_treatments)]
    return beta

In [304]:
# Price vector pi(i,t) = alpha(t) * pte(i,t) + beta(t). Dimensions num_subjects * num_treatments
def get_price_matrix(alpha, beta):
    price_matrix = [[(alpha[index] * pte_t + beta[index]) for index, pte_t in enumerate(pte)] for pte in pte_matrix]
    price_matrix = np.array(price_matrix)
    #print "get_price_matrix: Price matrix:", price_matrix
    return price_matrix

In [305]:
# Demand p*(i,t) matrix. Solve LP to get values. Dimensions num_subjects * num_treatments
def get_demand_matrix(price_matrix):
    prob_coefficient = [1] * num_treatments
    prob_threshold = 1
    # dummy first row
    demand_matrix = np.zeros(num_treatments)
    for i in range(num_subjects):
        # Constraints:
        # 1. <p*(i), pi(i)> <= b(i) for every subject i
        # 2. sum of all p*(t) = prob_threshold for every subject i
        coefficients = price_matrix[i]
        thresholds = budget_matrix[i]
        result = linprog(c=-wtp_matrix[i], 
                         # A_ub needs to be 2D array  
                         A_ub=np.stack((coefficients, coefficients)),
                         b_ub=np.stack((thresholds, thresholds)),
                         # A_eq needs to be 2D array
                         A_eq=np.stack((prob_coefficient, prob_coefficient)),
                         b_eq=np.stack((prob_threshold, prob_threshold)))
        demand_matrix = np.vstack((demand_matrix, result.x))
    # delete dummy first row
    demand_matrix = np.delete(demand_matrix, (0), axis=0)
    #print "get_demand_matrix: Demand matrix:", demand_matrix
    return demand_matrix

In [306]:
# Treatment_demand(t) = sum of demand(t) across all i. Dimensions 1 * num_treatments
def get_treatment_demand_matrix(demand_matrix):
    treatment_demand_matrix = np.zeros(num_treatments)
    for subject in range(num_subjects):
        for treatment in range(num_treatments):
            treatment_demand_matrix[treatment] += demand_matrix[subject, treatment]
    #print "get_treatment_demand_matrix: Treatment demand matrix:", treatment_demand_matrix
    return treatment_demand_matrix

In [307]:
# Excess_demand(t) = treatment_demand(t) - capacity(t). Dimensions 1 * num_treatments
def get_excess_demand_matrix(treatment_demand_matrix):
    excess_demand_matrix = treatment_demand_matrix - capacity_matrix
    #print "get_excess_demand_matrix: Excess demand matrix:", excess_demand_matrix
    return excess_demand_matrix

In [308]:
# Clearing error in market = sqrt(sum of excess_demand(t)^2 for every treatment t)
def get_clearing_error(excess_demand_matrix):
    # If demand is satisfied everywhere and total capacity > number of subjects, no clearing error
    if all(excess <= 0 for excess in excess_demand_matrix):
        print "get_clearing_error: Market clear, no clearing error!"
        return 0
    else:
        clearing_error = sqrt(sum([excess**2 for excess in excess_demand_matrix]))
        clearing_error = clearing_error / sum(capacity_matrix)
        print "get_clearing_error: Clearing error:", clearing_error
        return clearing_error

In [309]:
# Recalibrate alpha, beta values to set new prices
def get_alpha_new(alpha, excess_demand_matrix):
    alpha_new = alpha + excess_demand_matrix * alpha_scaling_factor
    for (i, a) in enumerate(alpha_new):
        if (a > 0):
            # alpha become +ve, so reset to random initialization
            alpha_new[i] = random.randint(-budget, 0)
    return alpha_new

def get_beta_new(beta, excess_demand_matrix):
    beta_new = beta + excess_demand_matrix * beta_scaling_factor
    return beta_new    

In [310]:
# Find market clearing price vector. The objective is to change alpha and beta values so that we reduce clearing error
def clear_market():
    
    # Initialize market prices and demand
    alpha = init_alpha()
    beta = init_beta()    
    price_matrix = get_price_matrix(alpha, beta)
    demand_matrix = get_demand_matrix(price_matrix)  
        
    excess_demand_matrix = get_excess_demand_matrix(get_treatment_demand_matrix(demand_matrix))
    clearing_error = get_clearing_error(excess_demand_matrix)
    
    # clearing_error_threshold = 0.01*(num_subjects)/(num_treatments)
    
    # clearing error is percentage of total capacity so we want the market to clear at 1%
    clearing_error_threshold = 0.01
    threshold_iterations = 10
    iterations = 0
    minimum_clearing_error = clearing_error
    alpha_star = 0
    beta_star = 0
    
    # Set new prices to clear market
    while True:
        if iterations > threshold_iterations:
            # new search start
            alpha = init_alpha()
            beta = init_beta()
            iterations = 0
            print "new search start"
            print alpha, beta         
        else:
            # continue down current search
            alpha = get_alpha_new(alpha, excess_demand_matrix)
            beta = get_beta_new(beta, excess_demand_matrix)
        
        price_matrix = get_price_matrix(alpha, beta)
        demand_matrix = get_demand_matrix(price_matrix)
        excess_demand_matrix = get_excess_demand_matrix(get_treatment_demand_matrix(demand_matrix))
        clearing_error = get_clearing_error(excess_demand_matrix)
        
        # Store parameter values for minimum clearing error
        if clearing_error < minimum_clearing_error:
            minimum_clearing_error = clearing_error
            alpha_star = alpha.copy()
            beta_star = beta.copy()
        # cleared the market! 
        if minimum_clearing_error < clearing_error_threshold:
            break
        iterations += 1
    
    print "Minimum clearing error:", minimum_clearing_error
    print "Alpha_star:", alpha_star
    print "Beta star:", beta_star
    return (minimum_clearing_error, alpha_star, beta_star)

In [315]:
alpha = init_alpha()
beta = init_beta()    
price_matrix1 = get_price_matrix(alpha, beta)
demand_matrix1 = get_demand_matrix(price_matrix)

print alpha, beta 
print "Prices: ", price_matrix1
print demand_matrix1

[-6, -4] [4, 5]
Prices:  [[-224 -339]
 [-164  -67]
 [ -80 -227]
 ..., 
 [-536  -51]
 [-350  -31]
 [-122 -187]]
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [316]:
alpha = init_alpha()
beta = init_beta()    
price_matrix2 = get_price_matrix(alpha, beta)
demand_matrix2 = get_demand_matrix(price_matrix)

print alpha, beta
print "Prices: ", price_matrix2
print demand_matrix2

[-9, -8] [-2, 1]
Prices:  [[-344 -687]
 [-254 -143]
 [-128 -463]
 ..., 
 [-812 -111]
 [-533  -71]
 [-191 -383]]
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [317]:
sum(demand_matrix1 - demand_matrix2)

array([ 0.,  0.])

In [270]:
# %timeit min_error, alpha_star, beta_star = clear_market()

min_error, alpha_star, beta_star = clear_market()
price_star = get_price_matrix(alpha_star, beta_star)
demand_star = get_demand_matrix(price_star)

[-89, -89] [55, 65]
get_clearing_error: Clearing error: 0.279229158657
get_clearing_error: Clearing error: 0.179004991466
get_clearing_error: Clearing error: 0.0804230156521
get_clearing_error: Clearing error: 0.278707329767
get_clearing_error: Clearing error: 0.184195852586
get_clearing_error: Clearing error: 0.103106838108
get_clearing_error: Clearing error: 0.278708302318
get_clearing_error: Clearing error: 0.18266032098
get_clearing_error: Clearing error: 0.094732258879


KeyboardInterrupt: 

In [271]:
demand_matrix1 - demand_matrix2

NameError: name 'demand_matrix1' is not defined

In [219]:
#print demand_star
for demands in demand_star:
    if demands[0] + demands[1] < 0.999999:
        print demands

In [221]:
stats.describe(demand_star)

DescribeResult(nobs=861, minmax=(array([ 0.32992467,  0.        ]), array([ 1.        ,  0.67007533])), mean=array([ 0.51984737,  0.48015263]), variance=array([ 0.08414563,  0.08414563]), skewness=array([ 1.05206162, -1.05206162]), kurtosis=array([-0.8923468, -0.8923468]))

In [226]:
def simulate(num_sim):
    sim_demand = []
    for i in range(num_sim):
        min_error, alpha_star, beta_star = clear_market()
        price_star = get_price_matrix(alpha_star, beta_star)
        demand_star = get_demand_matrix(price_star)
        sim_demand.append(demand_star)
    
    return sim_demand

In [252]:
%timeit sim_demand = simulate(20)

get_clearing_error: Clearing error: 0.279229158657
get_clearing_error: Clearing error: 0.115444133212
get_clearing_error: Clearing error: 0.0122809632205
get_clearing_error: Clearing error: 0.0100239359473
get_clearing_error: Clearing error: 0.00848900713081
Minimum clearing error: 0.00848900713081
Alpha_star: [-140.1706522  -84.8293478]
Beta star: [-134.6826088  187.6826088]
get_clearing_error: Clearing error: 0.279229158657
get_clearing_error: Clearing error: 0.138355717472
get_clearing_error: Clearing error: 0.0584478582669
get_clearing_error: Clearing error: 0.0344476107134
get_clearing_error: Clearing error: 0.0292554001089
get_clearing_error: Clearing error: 0.0224551982543
get_clearing_error: Clearing error: 0.0191640180004
get_clearing_error: Clearing error: 0.0153874672846
get_clearing_error: Clearing error: 0.0131244532778
get_clearing_error: Clearing error: 0.0107470482063
get_clearing_error: Clearing error: 0.00914759114916
Minimum clearing error: 0.00914759114916
Alpha_sta